# Instruções

A partir dessas instruções espero que se torne fácil o processo de criar uma macro seguindo esses passos.  
Recomendo que leia todas as instruções antes de verificar os exemplos e 

## Primeiros Passos

A não ser que você tenha conhecimento prévio de programação, recomendo que sejam executadas todas as células antes que comecem a utilizar o caderno. Isto fará com que não haja nenhuma dependência não instalada ou função não declarada.  
Para isso recomendo que clique em Kernel, entre *Cell* e *Widjets*, na barra abaixo do logo do Jupyter e, em seguida, *Restart & Run All*.  

## Entendendo os Exemplos

Além dos exemplos padrão, que utilizarei para demonstrar o funcionamento do programa, também deixarei comentados templates de chamada de função, que vocês utilizarão para criar suas próprias macros!

Um exemplo consiste em uma chamada de função com os valores já postos que retorna um resultado provavelmente muito próximo dos seus próprios resultados. Para executar um exemplo vá até a célula onde ele está e aperte CTRL+Enter, o resultado aparecerá abaixo da célula

## Entendendo os Templates

Os templates estão aqui para guiar vocês a montarem suas próprias chamas de funções para criar macros pras suas fichas. Templates estão organizados da seguinte forma:

```cria_descricao("{tipo_acao}", "{id_acao}")```  
```cria_template_rolagem("{tipo_acao}", "{id_acao}")```  
-> Para ações Habilidade e Poder  

```cria_descricao("spell", "{id_acao}", circulo="{circulo_magia}")```   
```cria_template_rolagem("spell", "{id_acao}", circulo="{circulo_magia}")```  
-> Para ações de Magia

```cria_descricao("attack", "{id_acao}", critico="{tipo_critico}")```   
```cria_template_rolagem("attack", "{id_acao}", critico="{tipo_critico}")```  
-> Para ações de Ataque

Os tipo de ação aceita são:
    
    * ability
    * attack
    * power
    * spell
    
Os tipos de critico aceitos são:

    * DANO
        > Multiplicando o valor dos dados pelo multiplicador de crítico
    * DADO
        > Multiplicando o número de dados do ataque
    
Os circulos de magia vão de 1 até 5.
 
O ID da ação é o ID armazenado pelo Roll 20. Para acessá-lo, clique com o botão direito na linha que contem a acão para qual você deseja criar a macro e procurar a div com classe ```rep_item```. Em seguida copie o valor sob o atributo ```data-reprowid```, este é seu ID.



Recomendo, como antes, caso ainda esteja confuso sobre como acessar o ID da ação, que assista o vídeo https://youtu.be/dGK5eYLgz3E?t=549, do Vinícius silva para mais informações.
PS: Isso é bem mais simples do que parece, e já posicionei o vídeo no momento que ele fala como pegar esse ID, que é a única coisa necessária para nossos objetivos.
    
## Criando uma Macro

Para criar uma macro, realize as substituições indicadas no tópico anterior e role sua célula com a chamada de função apertando CTRL+Enter enquanto estiver com ela selecionada.


## Finalizações

Com o tempo vocês se tornarão mais familiarizados com os notebooks pra perceber que podem realizar pequenas alterações nos templates fornecidos são apenas sujestões e que vocês podem criar seus próprios templates.
Espero que consigam utilizar com facilidade esse notebook!

##### Que Nimb guie seus dados!

In [1]:
pip install regex typing

Note: you may need to restart the kernel to use updated packages.


In [2]:
import base

from functools import partial
from typing import Callable, Tuple

In [3]:
def pega_atributos (tipo_acao: str) -> list:
    if tipo_acao == "ataque":
        return [ "nome{}".format(tipo_acao), "descricao{}".format(tipo_acao) ]
    else:
        return [ "name{}".format(tipo_acao), "{}description".format(tipo_acao) ]
    
def pega_funcao (tipo_funcao: int, tipo_acao: str, circulo: str, critico: base.CriticalTypes = None) -> Callable:
    if tipo_acao == "ability":
        return base.rpt_ability if tipo_funcao == 1 else base.create_ability_template
    elif tipo_acao == "power":
        return base.rpt_power if tipo_funcao == 1 else base.create_power_template
    elif tipo_acao == "spell":
        return (
            partial(base.rpt_spell, spell_circle=circulo)
            if tipo_funcao == 1
            else partial(base.create_spell_template, spell_circle=circulo)
        )
    elif tipo_acao == "attack":
        if critico is None:
            return base.rpt_power if tipo_funcao == 1 else base.create_attack_template
        else:
            return (
                partial(base.rpt_attack, critical=critico)
                if tipo_funcao == 1
                else partial(base.create_attack_template, critical=critico)
            )
    
    return None
        
def repete_acao (tipo_acao: str, circulo: str) -> Tuple[Callable, list]:
    atributos = pega_atributos(tipo_acao)
    funcao = pega_funcao(1, tipo_acao, circulo)
    
    return funcao, atributos

def macroniza_acao (tipo_acao: str, circulo: str, critico: base.CriticalTypes) -> Callable:
    funcao = pega_funcao(0, tipo_acao, circulo, critico)        
    return funcao

In [4]:
def cria_descricao (tipo_acao: str, id_acao: str, circulo: str = "1") -> str:
    descreve, atributos = repete_acao(tipo_acao, circulo)
    if descreve is None:
        print("Tipo Inválido de Ação: Ações válidas (ability, attack, power, spell)")
        return "TIPO INVÁLIDO DE AÇÃO"
    
    descricao = base.describe(*descreve(id_acao, rpt_attrs=atributos))
    
    print(descricao)
    return descricao

In [5]:
def cria_template_rolagem (tipo_acao: str, id_acao: str, circulo: str = "1", critico: str = "DANO") -> str:
    funcao = macroniza_acao(
        tipo_acao, str(circulo), base.CriticalTypes.DAMAGE if critico == "DANO" else base.CriticalTypes.DICE
    )
    if funcao is None:
        print("Tipo Inválido de Ação: Ações válidas (ability, attack, power, spell)")
        return "TIPO INVÁLIDO DE AÇÃO"
    
    template_rolagem = funcao(id_acao)
    
    print(template_rolagem)
    return funcao(id_acao)

## Início dos Templates

Caso não tenha ficado claro o funcionamento dos templates, recomendo fortemente que olhe os exemplos, na próxima seção, isto tornará muito mais fácil a compreensão.

Remova os # do início da linha que deseja e realize as alterações indicadas em ***Entendendo os Templates***. <br>
Em seguida pressione CTRL+Enter para receber o resultado da função.

In [19]:
#desc = cria_descricao("{tipo_acao}", "{id_acao}")
#temp_rol = cria_template_rolagem("{tipo_acao}", "{id_acao}")

In [7]:
#desc = cria_descricao("spell", "{id_acao}", circulo="{circulo_magia}")
#temp_rol = cria_template_rolagem("spell", "{id_acao}", circulo="{circulo_magia}")

In [8]:
#desc = cria_descricao("attack", "{id_acao}", critico="{tipo_critico}")
#temp_rol = cria_template_rolagem("attack", "{id_acao}", critico="{tipo_critico}")

## Início dos Exemplos

In [9]:
desc = cria_descricao("ability", "-MWol3wUlwt1di-xQWnZ")

**@{repeating_abilities_-MWol3wUlwt1di-xQWnZ_nameability}**: @{repeating_abilities_-MWol3wUlwt1di-xQWnZ_abilitydescription}


In [10]:
desc = cria_descricao("power", "-MWokal-6PQ7hFQQITAE")

**@{repeating_powers_-MWokal-6PQ7hFQQITAE_namepower}**: @{repeating_powers_-MWokal-6PQ7hFQQITAE_powerdescription}


In [11]:
temp_rol = cria_template_rolagem("attack", "-MX-d2s12Nz2i6J6qbQY", critico="DANO") # Multiplicando o dano dos dados

&{template:t20-attack}{{character=@{character_name}}}{{attackname=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_nomeataque}}}{{attackroll=[[1d20cs>@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_margemcriticoataque}+[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquepericia}]]+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_bonusataque}]]}} {{damageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}} {{criticaldamageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}*@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_multiplicadorcriticoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}}{{typeofdamage=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquetipodedano}}}{{description=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquedescricao}}}


In [12]:
temp_rol = cria_template_rolagem("attack", "-MX-d2s12Nz2i6J6qbQY", critico="DADO") # Multiplicando o número de dados

&{template:t20-attack}{{character=@{character_name}}}{{attackname=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_nomeataque}}}{{attackroll=[[1d20cs>@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_margemcriticoataque}+[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquepericia}]]+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_bonusataque}]]}} {{damageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}} {{criticaldamageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danocriticoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}}{{typeofdamage=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquetipodedano}}}{{description=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquedescricao}}}


In [13]:
temp_rol = cria_template_rolagem("spell", "-MWojXgYrdLO17N5ljE8", circulo="1")

&{template:spell}{{character=@{character_name}}}{{spellname=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_namespell}}}{{type=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spelltipo}}}{{execution=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellexecucao}}}{{duration=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellduracao}}}{{range=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellalcance}}}{{targetarea=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellalvoarea}}}{{resistance=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellresistencia}}}{{description=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spelldescription}}}{{cd=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellcd}}}


In [14]:
temp_rol = cria_template_rolagem("spell", "-MX-h-dXicmUNW88uEC-", circulo="2")

&{template:spell}{{character=@{character_name}}}{{spellname=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_namespell}}}{{type=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spelltipo}}}{{execution=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellexecucao}}}{{duration=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellduracao}}}{{range=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellalcance}}}{{targetarea=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellalvoarea}}}{{resistance=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellresistencia}}}{{description=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spelldescription}}}{{cd=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellcd}}}


In [15]:
# Exemplo com copia, rodo por padrão para não sobrescrever o CTRL+C do usuário
#temp_rol = cria_template_rolagem("spell", "-MX-h-dXicmUNW88uEC-", circulo="2")